# VariantSpark-k Small Hipster
This is our HPO notebook

# Imports

In [2]:
import shlex
from subprocess import Popen, PIPE

import numpy as np
import time 

ModuleNotFoundError: No module named 'numpy'

# Deal with prechecks

In [3]:
%%bash

set -e

MASTER=https://kubernetes.default.svc:443
INPUT_BUCKET=variant-spark

function fatal_error () {
	echo "ERROR: $1" 1>&2
	exit 1
}

if [ -z ${MASTER+x} ];
    then
        echo "You must set the MASTER environment variable to a kubernetes API endpoint";
        echo "Example: https://ABC.sk1.us-west-2.eks.amazonaws.com:443"
        exit 1
fi

if [ -z ${INPUT_BUCKET+x} ];
    then
        echo "You must set the INPUT_BUCKET environment variable to a bucket containing input data";
        echo "Example: variant-spark-k-storage"
        exit 1
fi

[[ $(type -P "spark-submit") ]] || fatal_error  "\`spark-submit\` cannot be found. Please make sure it's on your PATH."


# Run Grid search

In [4]:
'''
Sudo code
We have a grid of values (trees or batches)
We iterate through the grid of values and find which takes the least amount of time
We have that configuration as a variable 
Once iterated we output that value

'''

# Process file system params
MASTER="https://kubernetes.default.svc:443"
INPUT_BUCKET="variant-spark"
input_file = "s3://{}/datasets/hipterIndex/hipster.vcf".format(INPUT_BUCKET)
input_features = "s3://{}/datasets/hipterIndex/hipster_labels.txt".format(INPUT_BUCKET)
feature_column = "label"

# Process run params
n_iterations = 10
upper_bound = 10000
lower_bound = 100

# builds a list of evenly spaces values for the tree_val variable
n_trees = np.linspace(lower_bound, upper_bound, num=n_iterations)
n_trees = [int(x) for x in n_trees]

least_time_val = np.inf
param_dict = {}

for tree_val in n_trees:
    
    total_time = np.inf
    
    time_start = time.time()
    
    MASTER="https://kubernetes.default.svc:443"
    INPUT_BUCKET="variant-spark"
    
    input_string = "spark-submit \
        --class au.csiro.variantspark.cli.VariantSparkApp \
        --driver-class-path ./conf \
        --master k8s://${0} \
        --deploy-mode cluster \
        --name VariantSparkSmallHipsterHPO \
        --conf spark.kubernetes.authenticate.driver.serviceAccountName=spark \
        --conf spark.executor.instances=12 \
        --conf spark.executor.memory=2g \
        --conf spark.kubernetes.container.image=jamesrcounts/variantspark:002 \
        --jars http://central.maven.org/maven2/org/apache/hadoop/hadoop-aws/2.7.3/hadoop-aws-2.7.3.jar,http://central.maven.org/maven2/com/amazonaws/aws-java-sdk/1.7.4/aws-java-sdk-1.7.4.jar,http://central.maven.org/maven2/joda-time/joda-time/2.9.9/joda-time-2.9.9.jar \
        local:///opt/spark/jars/variant-spark_2.11-0.2.0-SNAPSHOT-all.jar importance \
            -if {2} \
            -ff {3} \
            -fc {4} \
            -v \
            -rn {1} \
            -rbs 28 \
            -ro \"$@\"".format(MASTER, tree_val, input_file, input_features, feature_column)
    
    args = shlex.split(input_string)
    process = Popen(args, stdout=PIPE, stderr=PIPE)
    #stdout, stderr = process.communicate()
    
    time_stop = time.time()
    total_time = time_stop - time_start
    
    print("process finished at {} seconds using {} trees".format(total_time, tree_val))
    
    if (total_time < least_time_val):
        least_time_val = total_time
        
    param_dict[total_time] = tree_val
    
print("The ntree value that took the least amount of time was {} at {} seconds".format(param_dict[least_time_val], least_time_val))

NameError: name 'np' is not defined